In [18]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

import pandas as pd
import glob
import matplotlib.pyplot as plt

## Google Colab

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


# 先行研究

## 発話エンコーダ設定

In [107]:
utter_embedding = tf.keras.layers.Embedding(1000, 300)
utter_gru = tf.keras.layers.GRU(512)

## 文脈エンコーダ設定

In [ ]:
context_gru = tf.keras.layers.GRU(513)

## 対話行為エンコーダ設定

In [ ]:
dialog_embedding = tf.keras.layers.Embedding(1000, 100)
dialog_gru = tf.keras.layers.GRU(128)

## 分類器設定

## 損失関数設定

## 最適化関数設定

## 実行(前処理)

## filepath(colab)

In [45]:
#train_data = "drive/My Drive/研究/swda_data/train_set.txt"
#test_data = "../Switchboard-Corpus/swda_data/test_set.txt"
#val_data = "drive/My Drive/研究/swda_data/val_set.txt"

## filepath(local)

In [2]:
train_data = "../Switchboard-Corpus/swda_data/train_set.txt"
test_data = "../Switchboard-Corpus/swda_data/test_set.txt"
val_data = "../Switchboard-Corpus/swda_data/val_set.txt"

## パラメータ設定

In [3]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

## ファイル取得

In [4]:
train_dataset = []
f = open(train_data, "r", encoding='utf-8')
for row in f:
    train_dataset.append(row.strip())
f.close()

In [5]:
val_dataset = []
f = open(val_data, "r", encoding='utf-8')
for row in f:
    val_dataset.append(row.strip())
f.close()

In [6]:
test_dataset = []
f = open(val_data, "r", encoding='utf-8')
for row in f:
    test_dataset.append(row.strip())
f.close()

## 発話者,発話,発話ラベル

In [7]:
utter_user = []
utter = []
utter_label = []
datasets = [train_dataset, val_dataset, test_dataset]

for d in datasets:
    for j in d:
        for i,v in enumerate(j.split("|")):
            if i == 0:
                utter_user.append(v)
            elif i==1:
                utter.append(v)
            else:
                utter_label.append(v)
print("finish")

finish


In [8]:
labels = set(utter_label)
utter_labels = []
for i in utter_label:
    for j,v in enumerate(labels):
        if i == v:
            utter_labels.append(j)
        

In [9]:
utter_labels[:10]

[35, 20, 0, 30, 11, 5, 38, 5, 5, 6]

In [63]:
len(utter_label),len(utter),len(utter_user)

(198934, 198934, 198934)

## データセット化

In [10]:
train_datasets = tf.data.Dataset.from_tensor_slices((utter, utter_labels))

In [11]:
text, label = next(iter(train_datasets.take(1)))
print(text)
print(label)

tf.Tensor(b'Okay.', shape=(), dtype=string)
tf.Tensor(35, shape=(), dtype=int32)


In [12]:
for train_example, train_label in train_datasets.take(1):
    print(train_example.numpy())
    print(train_label.numpy())

b'Okay.'
35


## トークナイザー

In [19]:
## ボキャブラリーリスト
vocabulary_set = set()
## トークナイザー
tokenizer = tfds.features.text.Tokenizer()

## 分かち書き
for text_tensor,_ in train_datasets:
    some_tokens = tokenizer.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)
    
## ボキャブラリーリスト作成
vocab_size = len(vocabulary_set)

In [20]:
print(vocab_size)

22042


In [23]:
i,v = next(iter(train_datasets))
print(i,v)

tf.Tensor(b'Okay.', shape=(), dtype=string) tf.Tensor(35, shape=(), dtype=int32)


## エンコード(トークンに変換する)

In [15]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [16]:
example_text = next(iter(train_datasets))[0].numpy()
print(example_text)

b'Okay.'


In [97]:
encoded_example = encoder.encode(example_text)
print(encoded_example)

[14576]


In [29]:
def encode(token, label):
    token = encoder.encode(token.numpy())
    return token, label

@tf.function
def tf_encoder(utter, label):
    encoded_text, label = tf.py_function(encode,[utter, label],[tf.int64, tf.int32])
    encoded_text.set_shape([None])
    label.set_shape([])
    return encoded_text, label

MAX_LENGTH = 40
def filter_max_length(x,y,max_length=MAX_LENGTH):
    return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

In [30]:
all_train_data_encode = train_datasets.map(tf_encoder)

In [32]:
i,v = next(iter(all_train_data_encode.take(1)))
print(i,v)

tf.Tensor([2981], shape=(1,), dtype=int64) tf.Tensor(35, shape=(), dtype=int32)


In [38]:
#all_train_data_encode = all_train_data_encode.filter(filter_max_length)

## 訓練データ
utter_train_data = all_train_data_encode.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
utter_train_data = utter_train_data.padded_batch(BATCH_SIZE)

## テストデータ
utter_test_data = all_train_data_encode.take(TAKE_SIZE)
utter_test_data = utter_test_data.padded_batch(BATCH_SIZE)

In [49]:
for ex in utter_train_data.take(1):
    print(ex)

(<tf.Tensor: shape=(64, 35), dtype=int64, numpy=
array([[  833,   782,   782, ...,     0,     0,     0],
       [ 6388,  5264,  1588, ...,     0,     0,     0],
       [ 2571, 20061,   983, ...,     0,     0,     0],
       ...,
       [20864, 21425,     0, ...,     0,     0,     0],
       [10064,     0,     0, ...,     0,     0,     0],
       [21426, 15040,     0, ...,     0,     0,     0]])>, <tf.Tensor: shape=(64,), dtype=int32, numpy=
array([11, 11, 26, 11, 40, 36, 37, 11, 40, 11,  6, 40, 11, 11, 11, 11, 11,
       40, 11, 11, 20,  6, 11, 11, 40, 40, 11, 11, 40, 11,  6, 26, 17, 11,
       11, 40, 27,  6, 11, 40, 32, 37,  8, 32,  6, 26, 40, 11, 11, 11,  1,
       40, 11, 30, 17, 11, 26, 11,  6,  2,  6, 17, 36, 40], dtype=int32)>)


In [40]:
a,b = next(iter(utter_test_data))
print(a[0])
print(b[1])

tf.Tensor(
[2981    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0], shape=(21,), dtype=int64)
tf.Tensor(20, shape=(), dtype=int32)


In [54]:
vocab_size += 1

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 300),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(41)
])

In [55]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [56]:
model.fit(utter_train_data, 
          epochs=3, 
          validation_data=utter_test_data,
          validation_steps=30)

Epoch 1/3
3031/3031 [==============================] - 2376s 784ms/step - loss: 0.9311 - accuracy: 0.7110 - val_loss: 0.8674 - val_accuracy: 0.7214
Epoch 2/3
3031/3031 [==============================] - 2252s 743ms/step - loss: 0.7605 - accuracy: 0.7526 - val_loss: 0.8407 - val_accuracy: 0.7380
Epoch 3/3
3031/3031 [==============================] - 2393s 789ms/step - loss: 0.6785 - accuracy: 0.7751 - val_loss: 0.8931 - val_accuracy: 0.7411


In [35]:
"""
train_data_set = utter_train_dataset.map(tf_encoder)
train_data_set = train_data_set.filter(filter_max_length)
train_data_set = train_data_set.cache()
train_data_set = train_data_set.padded_batch(BATCH_SIZE, padded_shapes=(100,100))
train_data_set.prefetch(tf.data.experimental.AUTOTUNE)
"""

'\ntrain_data_set = utter_train_dataset.map(tf_encoder)\ntrain_data_set = train_data_set.filter(filter_max_length)\ntrain_data_set = train_data_set.cache()\ntrain_data_set = train_data_set.padded_batch(BATCH_SIZE, padded_shapes=(100,100))\ntrain_data_set.prefetch(tf.data.experimental.AUTOTUNE)\n'

In [101]:
class UtterEncoder(tf.keras.Model):
    def __init__(self, vocabsize, embedding_dim, units, batch_size, output_dim):
        super(UtterEncoder, self).__init__()
        self.units = units
        self.batch_size = batch_size
        self.output_dim = output_dim
        self.embedding = tf.keras.layers.Embedding(vocabsize, embedding_dim)
        self.utter_gru = tf.keras.layers.GRU(self.units,
                                     return_sequences=True,
                                     return_state=True,
                                     recurrent_activation='sigmoid',
                                     recurrent_initializer='glorot_uniform')
        self.fc1 = tf.keras.layers.Dense(self.units, activation='relu')
        self.fc2 = tf.keras.layers.Dense(self.output_dim)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, states = self.utter_gru(x, initial_state=hidden)
        output = tf.reshape(output,(-1, output.shape[2]))
        x = self.fc1(output)
        x = self.fc2(x)
        return x, states
        

## 発話エンコーダ

In [109]:
utter_encoder = UtterEncoder(vocab_size, 300, 512, 64, 64)

### 最適化関数

In [72]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

### 損失関数

In [97]:
def loss_function(real, preds):
    return tf.keras.losses.sparse_categorical_crossentropy(real, preds)

## 確認

In [106]:
utter_train_data

<PaddedBatchDataset shapes: ((None, None), (None,)), types: (tf.int64, tf.int32)>

In [108]:
y_true = [1, 2]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]

## 実行処理

In [74]:
EPOCHS = 3

In [110]:
for epochs in range(EPOCHS):
    hidden = utter_encoder.reset_states()
    
    for (batch, (inp, tar)) in enumerate(utter_train_data):
        with tf.GradientTape() as tape:
            predictions, hidden = utter_encoder(inp, hidden)
            target = tf.reshape(tar, (-1,))
            print(predictions)
            print(target)
            loss = loss_function(target, predictions)
            
        grads = tape.gradient(loss, utter_encoder.variables)
        optimizer.apply_gradients(zip(grads, utter_encoder.variables))
        if batch % 100 == 0:
              print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch+1,
                                                            batch,
                                                            loss))

tf.Tensor(
[[ 0.00913575 -0.0089182   0.00074195 ... -0.00252971 -0.00192541
  -0.01607111]
 [ 0.00273279 -0.02137799 -0.00433403 ... -0.00394433  0.00172155
  -0.01313791]
 [ 0.00308836 -0.0309902  -0.00698273 ... -0.00209094  0.00370451
  -0.01702399]
 ...
 [-0.00019921 -0.04444277 -0.00912311 ...  0.00434959  0.00228057
  -0.02517691]
 [-0.00019923 -0.04444281 -0.00912323 ...  0.00434965  0.00228072
  -0.02517692]
 [-0.00019923 -0.04444283 -0.00912329 ...  0.00434967  0.0022808
  -0.02517693]], shape=(2432, 41), dtype=float32)
tf.Tensor(
[11 11 11 40 40 11 26 26 40 11 40 26 26 37 26 37 40 20 11 11 37 37 32  6
 11 30  6 11 28 11 40 26 11 11 37  0 36 11 11 32 40 11 37 11 40  6 11 37
 36 11  3 40  2  8  4 11 11 11 11 37  6 12 20 27], shape=(64,), dtype=int32)


ValueError: Shape mismatch: The shape of labels (received (64,)) should equal the shape of logits except for the last dimension (received (2432, 41)).

In [ ]:
"""
class ContextEncoder(tf.keras.Model):
    def __init__(self, units, batch_size):
        super(ContextEncoder, self).__init__()
        self.units = units
        self.batch_size = batch_size
        self.context_gru = tf.keras.GRU(self.units,
                                     return_sequences=True,
                                     return_state=True,
                                     recurrent_activation='sigmoid',
                                     recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(self.units)
                                     
    def call(self, x, hidden):
        output, states = self.gru(x, initial_state=hidden)
        #output = tf.reshape(output,(-1, output.shape[2]))
        #x = self.fc(output)
        return output, states
        
context_encoder = ContextEncoder(513, 64)
"""

In [ ]:
class FeedFowardNetwork():
    def __init__(self, units):
        super(FeedFowardNetwork, self).__init__()
        self.units = units
        self.fc = tf.keras.layers.Dense(self.units)
    
    def call(self, output):
        x = self.fc(output)
        return x

### 実行処理

In [90]:
utter_train_data

<PaddedBatchDataset shapes: ((None, None), (None,)), types: (tf.int64, tf.int32)>

## 訓練データセット作成

### ウィンドウサイズ

In [59]:
window_size = 5

In [ ]:
for i in range(len(utter)):
    

### 実行モデル version

In [57]:
print("TensorFlow version: ", tf.__version__)

TensorFlow version:  2.3.0
